# Webscrapping de datos del sitio de compras de la Municipalidad de General Pueyrredón


In [239]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [156]:
# Definimos la URL para scrappear
url = 'https://appsvr.mardelplata.gob.ar/Consultas07/OrdenesDeCompra/OC/index.asp?fmANIO_CON=2022&fmJURISDICCION_CON=1110101000&fmTIPOCONT_CON=--&fmNRO_OC=&Consultar=Consultar' 

# Hacemos el request de la url, y luego lo metemos en BeautifulSoup
html = requests.get(url.strip(), headers={'User-Agent': 'Mozilla/5.0'}).text
soup = BeautifulSoup(html, "lxml")


In [193]:
# Generamos una lista vacía, que la vamos a utilizar para cargar los elementos scrappeados de la tabla
data = []

# Iteramos a través de todas las filas (tr) de tabla. Cada fila por la que navegamos, tiene columnas,
# por la que también tenemos que navegar para extraer los datos.
# Por lo que hacemos dos iteraciones, una por las filas, y en cada fila, por cada columna. Y ahi obtenemos
# el dato para agregar a la base

# Iteramos por las filas de la tabla (tr)
for row in soup.find_all('tr'):
    # De cada fila, obtenemos las columnas (td) con un find_all dentro de la fila row. Pero de las columnas, solo
    # buscamos las que tienen la clase "textocomun"
    columns = row.find_all('td', class_='textocomun')
    
    # Definimos una lista td para cargar toda la info que obtenemos de cada columna
    td = []
    
    # Iteramos por cada columna columns
    for cols in columns:
        # Eliminamos los espacios del comienzo y final de la cadena
        cols = cols.text.strip()
        # Agregamos cols a un lista que se llama td. Cuando termina el bucle cols, td es una lista 
        # donde cada elemento es el valor de una columna de UNA fila
        td.append(cols)
        
    # Fuera del bucle cols, tenemos td con la info de la fila. Tenemos que agregarlo a data, que es otra lista
    # donde cada elemento de la lista es una fila, con cada elementode esa segunda lista siendo el valor de cada
    # columna.
    # Si td no está vacío (sería una row vacía) o si no tiene 'Formulario de consulta' en la posición 0, 
    # agregamos a td a data
    if (len(td) != 0) and (td[0]!='Formulario de Consulta'):
        data.append(td) 

In [194]:
# En la variable columna definimos cuales serían los nombres de las columnas del dataframe.
# La primer columna se llama remove, porque la vamos a borrar porque es una columna vacía
columnas = ['remove', 'oc','fecha_emision','importe','proveedor','tipo_contratacion','observaciones','estado']
df = pd.DataFrame(data, columns = columnas)

In [195]:
# Eliminamos la columna remove
df = df[['oc', 'fecha_emision', 'importe', 'proveedor',
       'tipo_contratacion', 'observaciones', 'estado']]

In [196]:
df

,oc,fecha_emision,importe,proveedor,tipo_contratacion,observaciones,estado
0,1000,16/8/2022,"$338.400,00",BERTOLAMI MABEL ELENA,Compra Directa Varios Proveedores,Destino: Item 1: 8 matafuegos para el Archivo ...,Registrada
1,564,17/5/2022,"$22.000,00",AGUIRRE RICARDO OSVALDO,Compra Directa Varios Proveedores,LIBRE DE TODO GASTO ADICIONAL.,Registrada
2,147,18/2/2022,"$273.000,00",CAZOU RICARDO MARTIN,Concurso de precios,Expte 7895-6-2021 cpo 1 alc 2 cpo 1. Prorroga ...,Registrada
3,155,21/2/2022,"$13.000,00",ARIAS JUAN ANIBAL,Compra Directa Varios Proveedores,"carga, descarga y flete a cargo del proveedor....",Registrada
4,951,8/8/2022,"$2.880.000,00",CAZOU RICARDO MARTIN,Licitación Privada,Expte 3623 Dig. 8 Año 2022 Cpo. 1. Licitacion ...,Registrada
5,932,3/8/2022,"$26.273,40",RADIOGRAFICA OESTE SRL,Licitación Pública,Expediente Nº 2731-5-2022 Cpo.6 Licitación Púb...,Registrada
6,506,6/5/2022,"$75.000,00",FEUER DIEGO MANUEL,Compra Directa Varios Proveedores,PERIODO: DESDE LA NOTIFICACIÓN DE LA ORDEN DE ...,Registrada
7,45,25/1/2022,"$85.550,00",CURRA ANTONIO,Compra Directa Varios Proveedores,"CARGA, DESCARGA Y FLETE A CARGO DEL PROVEEDOR....",Registrada
8,427,22/4/2022,"$38.400,00",SUAREZ CENTURION GABRIEL ALEJANDRO MATIAS,Compra Directa Varios Proveedores,"CARGA, DESCARGA Y FLETE A CARGO DEL PROVEEDOR....",Registrada
9,871,15/7/2022,"$196.528,50",MALTHUS SA,Compra Directa Varios Proveedores,LUGAR DE ENTREGA: SEC CULTURA OLAVARRIA 2508 1...,Registrada


## Generalizamos el script para todas las dependencias

In [214]:
jurisdicciones = [1110101000,
                  1110111000]


In [215]:
def init_bs(jur):
    # Definimos la URL para scrappear, armándola con el codigo de jurisdicción
    url = 'https://appsvr.mardelplata.gob.ar/Consultas07/OrdenesDeCompra/OC/index.asp?fmANIO_CON=2022&fmJURISDICCION_CON=' + str(jur)+'&fmTIPOCONT_CON=--&fmNRO_OC=&Consultar=Consultar' 
    print(url)
    # Hacemos el request de la url, y luego lo metemos en BeautifulSoup
    html = requests.get(url.strip(), headers={'User-Agent': 'Mozilla/5.0'}).text
    soup = BeautifulSoup(html, "lxml")
    print('conectado a ' + str(jur))
    return soup

In [237]:
def scrapper(soup, jur, columnas):
    # Generamos una lista vacía, que la vamos a utilizar para cargar los elementos scrappeados de la tabla
    data = []
    # Iteramos a través de todas las filas (tr) de tabla. Cada fila por la que navegamos, tiene columnas,
    # por la que también tenemos que navegar para extraer los datos.
    # Por lo que hacemos dos iteraciones, una por las filas, y en cada fila, por cada columna. Y ahi obtenemos
    # el dato para agregar a la base
    # Iteramos por las filas de la tabla (tr)
    for row in soup.find_all('tr'):
        print(row)
        # De cada fila, obtenemos las columnas (td) con un find_all dentro de la fila row. Pero de las columnas, solo
        # buscamos las que tienen la clase "textocomun"
        columns = row.find_all('td', class_='textocomun')
        
        td = []

        # Iteramos por cada columna columns
        for cols in columns:
            
            # Eliminamos los espacios del comienzo y final de la cadena
            cols = cols.text.strip()
            # Agregamos cols a un lista que se llama td. Cuando termina el bucle cols, td es una lista 
            # donde cada elemento es el valor de una columna de UNA fila
            td.append(cols)
            td.append(jur)
        # Fuera del bucle cols, tenemos td con la info de la fila. Tenemos que agregarlo a data, que es otra lista
        # donde cada elemento de la lista es una fila, con cada elementode esa segunda lista siendo el valor de cada
        # columna.
        # Si td no está vacío (sería una row vacía) o si no tiene 'Formulario de consulta' en la posición 0, 
        # agregamos a td a data
        if (len(td) != 0) and (td[0]!='Formulario de Consulta'):
            data.append(td)
            print(data)
        print('scrappeado ' + str(jur))
        df = pd.DataFrame(data, columns = columnas)
        return df

In [238]:
columnas = ['remove', 'oc','fecha_emision','importe','proveedor','tipo_contratacion','observaciones','estado']
df = pd.DataFrame()
for jur in jurisdicciones:
    soup = init_bs(jur)
    data = scrapper(soup,jur, columnas)
    time.sleep(30)

https://appsvr.mardelplata.gob.ar/Consultas07/OrdenesDeCompra/OC/index.asp?fmANIO_CON=2022&fmJURISDICCION_CON=1110101000&fmTIPOCONT_CON=--&fmNRO_OC=&Consultar=Consultar
conectado a 1110101000
<tr>
<td align="center" valign="middle">
<table border="0" width="80%">
<tr>
<td align="center" valign="middle" width="62">
<p align="center"><img border="0" height="37" src="/img/xServerSQL.jpg.pagespeed.ic.kCtLVGw198.jpg" width="51"/></p></td>
<td align="left" valign="middle">
<p class="main_title">
<font face="VERDANA, ARIAL, HELVETICA" size="5">Atención!</font></p>
</td>
</tr>
</table>
<div align="center">
<center>
<table border="0" width="80%">
<tr>
<td align="center" bgcolor="#FF6600" height="5"></td>
</tr>
<tr>
<td align="center" class="noticia1">
<address class="subtitle">Lamentamos
informarle que el Servidor de Base de Datos al que usted desea acceder
esta temporalmente fuera de línea debido a tareas de resguardo y/o
mantenimiento, reintente la operación más tarde y sepa disculpar las mol

In [220]:
data

,remove,oc,fecha_emision,importe,proveedor,tipo_contratacion,observaciones,estado
